In [1]:
import os 
os.chdir('../')
%pwd

'/home/paladin/Downloads/Bixi-OD-Matrix-Prediction/Bixi-OD-Matrix-Prediction'

In [16]:
import sys
import numpy as np
import keras
from pathlib import Path
import pickle
from timeseriesPredictor.exception import CustomException

In [17]:
def load_pickle_file(path: Path):
    file= open(path, 'rb')
    obj = pickle.load(file)
    return obj

data = load_pickle_file('artifacts/data_transformation/od/train.pkl')
data.shape

(635, 64, 64, 1)

In [22]:
dat = data.reshape(635,64*64)

In [19]:
def split_sequence(sequence, lag, new_input=False):
        '''
        This function splits a given univariate sequence into
        multiple samples where each sample has a specified number
        of time steps and the output is a single time step.
        param new_input: If True it is used for predicting new input
        '''
        try:
                
            X, y = list(), list()
            for i in range(len(sequence)):
                # find the end of this pattern
                end_ix = i + lag
                # check if we are beyond the sequence               
                if end_ix > len(sequence)-1:
                    break
            # gather input and output parts of the pattern
                seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
                X.append(seq_x)
                y.append(seq_y)
            return np.array(X), np.array(y)
    
        except Exception as e:
            raise CustomException(e, sys)
    
def convert_to_supervised(dat, lag):
        '''
        This function takes a 2D sequennce, scales the array and splits
        a given multivariate sequence into multiple samples where each sample has a specified number
        of time steps. It returns multiple time steps as well as the scaler.
        param df (DataFrame): Bike sharing demand for each community over time
        param lag (int): History length or time lag
        '''
        
        try:
            if isinstance(dat, np.ndarray):
                pass
            else:
                dat = dat.values
            
            m, n = dat.shape
            # e.g., if lag = 7, BIXI demand of past 7*15 minutes
            X = np.zeros((m-lag,lag, n))
            Y = np.zeros((m-lag,n))

            for i in range(0,n):
                x, y = split_sequence(dat[:,i], lag)
                X[:,:,i] = x
                Y[:,i] = y
            return X, Y
    
        except Exception as e:
            raise CustomException(e, sys)   

In [36]:
X, y = convert_to_supervised(dat, lag = 7)
train_x = X.reshape(628, 7, 64,64, 1)
train_y = y.reshape(628, 64,64, 1)

In [38]:
n_steps, n_length, n_features = 7, 64, 64
n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
model = keras.models.Sequential()
model.add(keras.layers.ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(n_steps, n_length, n_features,1)))
model.add(keras.layers.Flatten())
model.add(keras.layers.RepeatVector(n_outputs))
model.add(keras.layers.LSTM(200, activation='relu', return_sequences=True))
model.add(keras.layers.TimeDistributed(keras.layers.Dense(100, activation='relu')))
model.add(keras.layers.TimeDistributed(keras.layers.Dense(1)))
model.compile(loss='mse', optimizer='adam')
 # fit network
model.fit(train_x, train_y, epochs=10, batch_size=32, verbose=0)


2023-08-14 16:39:46.563163: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 812646400 exceeds 10% of free system memory.
2023-08-14 16:39:46.835754: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 812646400 exceeds 10% of free system memory.
2023-08-14 16:39:47.099955: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 812646400 exceeds 10% of free system memory.
2023-08-14 16:39:49.696411: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 812646400 exceeds 10% of free system memory.
2023-08-14 16:39:49.805304: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 812646400 exceeds 10% of free system memory.


KeyboardInterrupt: 